#### Task 1 (deadline 19.02.2020 24:00)
* Find your name on the spreadsheet https://docs.google.com/spreadsheets/d/1RR2I6toCkebbGU1UK83HS70Ru_l0_o-nnZIHyiFB0No/edit?usp=sharing. In opposite of your name there are 24 words of hyponyms, you have to insert five corresponding hypernyms next to them. Examples of hyponyms and hyperonyms relationship you can find above in the current Jupiter notebook.
* Find for each pair of hyponyms and hypernyms a corresponding snippet of a text with their mentions. The source of the text can be any free resources, e.g., Wikipedia, Google, Yandex, others. You should save the snippets and their URLs within the lab2 folder in your NLP git-repo with .csv file-extension in a single file.

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

In [1]:
import pandas as pd
from yargy import or_, rule, Parser, and_, not_
from yargy.predicates import normalized, gram, is_capitalized, dictionary, eq
from yargy import Parser
from yargy.pipelines import morph_pipeline
from yargy.tokenizer import MorphTokenizer
from yargy.interpretation import fact, attribute
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [2]:
START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        eq('-'),
        eq('это')
    ),    
    rule(
        eq('являются')
    ),
    rule(
        eq('является')
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [3]:
def get_hyper(row):
    main_word = row['Hyponim']
    text = row['Text']
    
    HYPONYM = eq(utils.deaccent(main_word.lower()))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, END),
        rule(END, MID, END),
        rule(END, MID, MID, END),
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    # print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps



In [22]:
import pandas as pd

data = pd.read_csv('zh_batyrkhan_dataset.csv')
data2 = data[0:120]
data.head()

,Hyponim,Hypernim,Text,Link
0,КАВАЛЕРГАРД,Страж,КАВАЛЕРГАРДЫ (от франц. cavalier - всадник и g...,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...
1,КАВАЛЕРГАРД,Телохранитель,КАВАЛЕРГАРДЫ (от французского cavalier - всадн...,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...
2,КАВАЛЕРГАРД,Полк,Один из самых блестящих и привилегированных по...,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...
3,КАВАЛЕРГАРД,Кавалерия,Кавалергарды представляли собой элиту русской ...,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...
4,КАВАЛЕРГАРД,Эскадрон,Кавалергарды подоспели уже в критический момен...,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...


In [24]:
data2['dec_hyper'] = data2.apply(lambda row: get_hyper(row), axis=1)

КАВАЛЕРГАРД []
КАВАЛЕРГАРД []
КАВАЛЕРГАРД [['гвардеиская', 'тяжелая', 'кавалерия', '—', 'кавалергарды']]
КАВАЛЕРГАРД []
КАВАЛЕРГАРД []
КАВАЛЬКАДА [['кавалькада', 'группа', 'всадников']]
КАВАЛЬКАДА []
КАВАЛЬКАДА []
КАВАЛЬКАДА [['кавалькада', 'почетного', 'эскорта']]
КАВАЛЬКАДА [['кавалькада', 'почетного', 'эскорта']]
КАВЕРНА []
КАВЕРНА []
КАВЕРНА []
КАВЕРНА []
КАВЕРНА [['каверна', 'площадью']]
КАГЭБЭШНИК []
КАГЭБЭШНИК []
КАГЭБЭШНИК []
КАГЭБЭШНИК [['кагэбэшник', 'с']]
КАГЭБЭШНИК []
КАДЫК [['однои', 'из', 'функции', 'кадыка', 'является', 'защита', 'стенок', 'и', 'лобнои', 'части', 'гортани', 'щитовидным', 'хрящом']]
КАДЫК []
КАДЫК []
КАДЫК []
КАДЫК []
КАЖДЕНИЕ []
КАЖДЕНИЕ []
КАЖДЕНИЕ [['сжигание', 'в', 'специальном', 'сосуде', '—', 'кадиле', '—', 'смеси', 'благовонных', 'веществ', 'перед']]
КАЖДЕНИЕ []
КАЖДЕНИЕ []
КАЗЕИН [['казеин', 'является', 'одним', 'из']]
КАЗЕИН []
КАЗЕИН []
КАЗЕИН []
КАЗЕИН []
КАЙНОЗОЙ [['каинозои', '—', 'текущая', 'эра', 'геологическои', 'истории', 'земли']]
КАЙНОЗ

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
